In [90]:
import pandas as pd
from sqlalchemy import create_engine

In [91]:
main_df = pd.read_csv("Resources/netflix_titles.csv")
main_df = main_df.dropna()
main_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
5,s6,TV Show,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,"July 1, 2017",2016,TV-MA,1 Season,"International TV Shows, TV Dramas, TV Mysteries",A genetics professor experiments with a treatm...


In [92]:

cleaned_df = main_df.rename(columns={'listed_in':'genre', 'cast':'actor', "release_year": "year"})
cleaned_df
                   

,show_id,type,title,director,actor,country,date_added,year,rating,duration,genre,description
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
5,s6,TV Show,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,"July 1, 2017",2016,TV-MA,1 Season,"International TV Shows, TV Dramas, TV Mysteries",A genetics professor experiments with a treatm...
...,...,...,...,...,...,...,...,...,...,...,...,...
7778,s7779,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
7780,s7781,Movie,Zoo,Shlok Sharma,"Shashank Arora, Shweta Tripathi, Rahul Kumar, ...",India,"July 1, 2018",2018,TV-MA,94 min,"Dramas, Independent Movies, International Movies",A drug dealer starts having doubts about his t...
7781,s7782,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...


In [93]:
# Updated ERD
# Show
# -
# show_id INT PK
# title VARCHAR(255)
# rating VARCHAR(255)
# description VARCHAR(255)
# director VARCHAR(255)
# type VARCHAR(255)
# date_added VARCHAR(255)
# year INT


# Actors
# -
# actor_id INT PK
# actor VARCHAR(255)
# show_id FK - Show.show_id

# Country
# -
# country_id INT PK
# country VARCHAR(255)
# show_id FK - Show.show_id

# Genre
# -
# genre_id INT PK
# genre VARCHAR(255)
# show_id FK - Show.show_id

In [94]:
# Create table for shows
show_df = cleaned_df[["show_id", "title", "director", "rating", "type", "date_added", "year", "description", ]]
show_df.head()

,show_id,title,director,rating,type,date_added,year,description
1,s2,7:19,Jorge Michel Grau,TV-MA,Movie,"December 23, 2016",2016,After a devastating earthquake hits Mexico Cit...
2,s3,23:59,Gilbert Chan,R,Movie,"December 20, 2018",2011,"When an army recruit is found dead, his fellow..."
3,s4,9,Shane Acker,PG-13,Movie,"November 16, 2017",2009,"In a postapocalyptic world, rag-doll robots hi..."
4,s5,21,Robert Luketic,PG-13,Movie,"January 1, 2020",2008,A brilliant group of students become card-coun...
5,s6,46,Serdar Akar,TV-MA,TV Show,"July 1, 2017",2016,A genetics professor experiments with a treatm...


In [95]:
# Create table for actors and transform
actor_df = cleaned_df[["actor", "show_id"]]

actor_df.actor = actor_df.actor.str.split(', ')
actor_df = actor_df.explode('actor').reset_index(drop=True)

actor_df

/Users/petertran/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,actor,show_id
0,Demián Bichir,s2
1,Héctor Bonilla,s2
2,Oscar Serrano,s2
3,Azalia Ortiz,s2
4,Octavio Michel,s2
...,...,...
37941,Manish Chaudhary,s7784
37942,Meghna Malik,s7784
37943,Malkeet Rauni,s7784
37944,Anita Shabdish,s7784


In [96]:
# Create table for countries and transform
country_df = cleaned_df[["country", "show_id"]]

country_df.country = country_df.country.str.split(', ')
country_df = country_df.explode('country').reset_index(drop=True)

country_df

/Users/petertran/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,country,show_id
0,Mexico,s2
1,Singapore,s3
2,United States,s4
3,United States,s5
4,Turkey,s6
...,...,...
6135,Czech Republic,s7783
6136,United Kingdom,s7783
6137,Denmark,s7783
6138,Netherlands,s7783


In [ ]:
# Create table for genres and transform
genre_df = cleaned_df[["genre", "show_id"]]

country_df.country = country_df.country.str.split(', ')
country_df = country_df.explode('country').reset_index(drop=True)

country_df


In [52]:

# cleaned_df.actors = cleaned_df.actors.str.split(', ')
# cleaned_df.genre = cleaned_df.genre.str.split(', ')
# cleaned_df.country = cleaned_df.country.str.split(', ')

# cleaned_df = cleaned_df.explode('actors').reset_index(drop=True)
# cleaned_df = cleaned_df.explode('genre').reset_index(drop=True)
# cleaned_df = cleaned_df.explode('country').reset_index(drop=True)

# cleaned_df

,show_id,type,title,director,actors,country,date_added,release_year,rating,duration,genre,description
0,s2,Movie,7:19,Jorge Michel Grau,Demián Bichir,Mexico,"December 23, 2016",2016,TV-MA,93 min,Dramas,After a devastating earthquake hits Mexico Cit...
1,s2,Movie,7:19,Jorge Michel Grau,Demián Bichir,Mexico,"December 23, 2016",2016,TV-MA,93 min,International Movies,After a devastating earthquake hits Mexico Cit...
2,s2,Movie,7:19,Jorge Michel Grau,Héctor Bonilla,Mexico,"December 23, 2016",2016,TV-MA,93 min,Dramas,After a devastating earthquake hits Mexico Cit...
3,s2,Movie,7:19,Jorge Michel Grau,Héctor Bonilla,Mexico,"December 23, 2016",2016,TV-MA,93 min,International Movies,After a devastating earthquake hits Mexico Cit...
4,s2,Movie,7:19,Jorge Michel Grau,Oscar Serrano,Mexico,"December 23, 2016",2016,TV-MA,93 min,Dramas,After a devastating earthquake hits Mexico Cit...
...,...,...,...,...,...,...,...,...,...,...,...,...
113131,s7784,Movie,Zubaan,Mozez Singh,Anita Shabdish,India,"March 2, 2019",2015,TV-14,111 min,International Movies,A scrappy but poor boy worms his way into a ty...
113132,s7784,Movie,Zubaan,Mozez Singh,Anita Shabdish,India,"March 2, 2019",2015,TV-14,111 min,Music & Musicals,A scrappy but poor boy worms his way into a ty...
113133,s7784,Movie,Zubaan,Mozez Singh,Chittaranjan Tripathy,India,"March 2, 2019",2015,TV-14,111 min,Dramas,A scrappy but poor boy worms his way into a ty...
113134,s7784,Movie,Zubaan,Mozez Singh,Chittaranjan Tripathy,India,"March 2, 2019",2015,TV-14,111 min,International Movies,A scrappy but poor boy worms his way into a ty...


In [35]:
# # Looking at values within columns

# unique_combo = main_df["listed_in"].unique()
# genre_list = []
# for unique in unique_combo:
#     genre_split = unique.split(', ')
#     for genre in genre_split:
#         if genre not in genre_list:
#             genre_list.append(gen|re)
# len(genre_list)

42

In [ ]:
# movie_titles
# -
# ID VARCHAR PK
# show_name VARCHAR
# rating_ID INTEGER FK >- rating.rating_ID

# rating
# -
# rating_ID INTEGER PK
# rating_description VARCHAR

# listed_in
# -
# listed_in_ID INTEGER
# listed_in_description VARCHAR
# many to many

# info
# -
# ID VARHCAR FK >- movie_titles.ID
# listed_in_ID INTEGER FK >- listed_in.listed_in_ID
# type_ID VARCHAR FK >- type.type_ID
# date_added DATE
# rel_yr VARCHAR
# duration VARCHAR
# description VARCHAR

# country VARHCAR

# type
# -
# type_ID INTEGER PK
# type_desc VARCHAR
# not necessary

# cast
# -
# cast_ID INTEGER PK FK >- crew.cast_ID
# cast VARHCAR
# many to many

# director
# -
# director_ID INTEGER PK
# director VARCHAR
# not necessary

# crew
# -
# ID VARCHAR FK >- movie_titles.ID
# cast_ID INTGER FK
# director_ID INTEGER FK >- director.director_ID
# not necessary


In [25]:
movie_titles_df = main_df[["show_id", "title"]]
movie_titles_df

,show_id,title
1,s2,7:19
2,s3,23:59
3,s4,9
4,s5,21
5,s6,46
...,...,...
7778,s7779,Zombieland
7780,s7781,Zoo
7781,s7782,Zoom
7782,s7783,Zozo


In [27]:
ratings_df = main_df["rating"]
ratings_df

1       TV-MA
2           R
3       PG-13
4       PG-13
5       TV-MA
        ...  
7778        R
7780    TV-MA
7781       PG
7782    TV-MA
7783    TV-14
Name: rating, Length: 4808, dtype: object

In [29]:
cast = main_df[["cast"]]
# iterrows, split cast column on comma, new list from split, loop or list comprehension to create new dictionary
cast

,show_id,cast
1,s2,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ..."
2,s3,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ..."
3,s4,"Elijah Wood, John C. Reilly, Jennifer Connelly..."
4,s5,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar..."
5,s6,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan..."
...,...,...
7778,s7779,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ..."
7780,s7781,"Shashank Arora, Shweta Tripathi, Rahul Kumar, ..."
7781,s7782,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma..."
7782,s7783,"Imad Creidi, Antoinette Turk, Elias Gergi, Car..."
